# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Importing the dataset

In [2]:
dataset = pd.read_excel('Peak_Incident_Pressure.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_mass        576 non-null    float64
 1   Standoff_distance  576 non-null    int64  
 2   Peak_Pressure      576 non-null    float64
 3   Peak_time          576 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 18.1 KB


In [3]:
y = dataset['Peak_Pressure']
X = dataset.drop(['Peak_time','Peak_Pressure'], axis=1)
print(X.shape, y.shape)

(576, 2) (576,)


In [4]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Hyperparameter tuning 1 - layers, neurons, activation function

In [6]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ 2
    nodes = first_layer_nodes
    for i in range(1, 4):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [7]:
from tensorflow.keras.optimizers import Adam
def create_model(first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(first_layer_nodes, last_layer_nodes)
    for i in range(1, 4):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, epochs = 500, batch_size = 50)

C:\Users\z5379606\AppData\Local\Temp\ipykernel_44448\867054689.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, epochs = 500, batch_size = 50)


In [8]:
activation_funcs = ['relu', 'softplus', 'leaky_relu'] 
param_grid = dict(first_layer_nodes = [150, 200, 250],
                  last_layer_nodes = [80, 100, 120],
                  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
9/9 [==============================] - 1s 4ms/step - loss: 2171088797696.0000 - mae: 506913.4688
Epoch 2/500
9/9 [==============================] - 0s 3ms/step - loss: 2171062714368.0000 - mae: 506882.5938
Epoch 3/500
9/9 [==============================] - 0s 3ms/step - loss: 2171004780544.0000 - mae: 506812.0000
Epoch 4/500
9/9 [==============================] - 0s 3ms/step - loss: 2170860863488.0000 - mae: 506654.8438
Epoch 5/500
9/9 [==============================] - 0s 3ms/step - loss: 2170615496704.0000 - mae: 506321.4688
Epoch 6/500
9/9 [==============================] - 0s 3ms/step - loss: 2170125025280.0000 - mae: 505716.6875
Epoch 7/500
9/9 [==============================] - 0s 3ms/step - loss: 2169200836608.0000 - mae: 504640.3438
Epoch 8/500
9/9 [==============================] - 0s 3ms/step - loss: 2167851712512.0000 - mae: 502762.4062
Epoch 9/500
9/9 [==============================] - 0s 3ms/step - loss: 2165553758208.0000 - mae: 499688.6875
Epoch 10/500
9/9 [=

### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [9]:
# Function to create model, required for KerasRegressor
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=250, input_shape=(X_train.shape[1],), activation='relu'))
 model.add(Dense(units=185, activation='relu'))
 model.add(Dense(units=120, activation='relu'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [14]:
from scikeras.wrappers import KerasRegressor
# create model
model = KerasRegressor(model=create_model)

# define the grid search parameters
batch_size = [20,30,40]
epochs = [300, 400, 500]
optimizer = ['RMSprop', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/400
14/14 [==============================] - 1s 12ms/step - loss: 2171078705152.0000 - mae: 506907.7812
Epoch 2/400
14/14 [==============================] - 0s 11ms/step - loss: 2171015659520.0000 - mae: 506834.8125
Epoch 3/400
14/14 [==============================] - 0s 11ms/step - loss: 2170802536448.0000 - mae: 506606.2812
Epoch 4/400
14/14 [==============================] - 0s 12ms/step - loss: 2170299482112.0000 - mae: 505979.0312
Epoch 5/400
14/14 [==============================] - 0s 11ms/step - loss: 2169104367616.0000 - mae: 504489.2812
Epoch 6/400
14/14 [==============================] - 0s 11ms/step - loss: 2166763683840.0000 - mae: 501145.0000
Epoch 7/400
14/14 [==============================] - 0s 13ms/step - loss: 2161871290368.0000 - mae: 494916.2500
Epoch 8/400
14/14 [==============================] - 0s 13ms/step - loss: 2154012344320.0000 - mae: 483721.6562
Epoch 9/400
14/14 [==============================] - 0s 12ms/step - loss: 2140234448896.0000 - mae: 4658